In [260]:
# Import packages we'll need later on in this tutorial
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.ticker as mtick
import matplotlib.cm as cm
import math
import plotly.graph_objects as go
from collections import OrderedDict
from matplotlib.lines import Line2D  # Import for creating custom legend markers
import json
import copy

# Import BW25 packages
import bw2data as bd
import bw2io as bi
import bw2calc as bc
import bw2analyzer as bwa
import brightway2 as bw 
from bw2calc import LeastSquaresLCA

In [261]:
from standards import *
from LCA_initialization import *

In [262]:
save_dir = results_folder('Results')

Folder name Results created


In [263]:
# def LCA_initialization(name, db, flows):
#     db_consq = 'Consequential'
#     bd.projects.set_current(name)

#     bi.bw2setup()
#     eidb = bd.Database(db)
#     eidb_consq = bd.Database(db_consq)

#     # Corrected dictionary comprehension
#     procces_keys = {key: None for key in flows}

#     size = len(flows)

#     # Combine both databases into a single list
#     all_acts = list(eidb) + list(eidb_consq)

#     for act in all_acts:
#         #print(act)
#         for proc in range(size):
#             if act['name'] == flows[proc]:
#                 procces_keys[flows[proc]] = act['code']
#                 #print(act['name'], act['code'])
                
#     process = []
#     key_counter = 0

#     for key, item in procces_keys.items():
#         # Locate the specific process
#         try:
#             if eidb.get(item) in eidb:
#                 process.append(eidb.get(item))
#                 print('DB', eidb.get(item))
#             else:
#                 process.append(eidb_consq.get(item))
#                 print('CONSQ', eidb_consq.get(item))
#         except KeyError:
#             print(f"Process with key '{item}' not found in the database '{eidb}'")
#             process = None
        
#         key_counter += 1

#     products_list = []

#     if process:
#         for proc in process:
#             # Loop through the exchanges and filter for products
#             for exc in proc.exchanges():
#                 if exc['type'] == 'production':
#                     # Add the product (output) to the list
#                     products_list.append(exc.input)


#     # Initialize an empty list to store the linked processes
#     linked_processes_list = []

#     if process:
#         for proc in process:
#             # Loop through the exchanges and extract linked process keys
#             for exc in proc.exchanges():
#                 # Add the linked process (input) to the list
#                 linked_processes_list.append(exc.input)

#             # Remove duplicates by converting to a set and back to a list
#             linked_processes_list = list(set(linked_processes_list))

#         # Display the list of linked processes
#         proc_keys = {}
#         name_keys = {}

#         for linked_process in linked_processes_list:
#             # Initialize the list for this database if it doesn't exist
#             if linked_process[0] not in proc_keys:
#                 proc_keys[linked_process[0]] = []
#                 name_keys[linked_process[0]] = []
#             # Append the process key to the list
#             proc_keys[linked_process[0]].append(linked_process[1])
#             name_keys[linked_process[0]].append(linked_process)

#     # List all methods containing 'EF v3.1 EN15804'
#     all_methods = [m for m in bw.methods if 'EF v3.1 EN15804' in str(m)]

#     # Filter out methods that contain "climate change:" in method[1]
#     filtered_methods = [method for method in all_methods if "climate change:" not in method[1]]

#     # Print the methods that were removed
#     removed_methods = [method[1] for method in all_methods if "climate change:" in method[1]]

#     impact_category = filtered_methods
    
#     plot_x_axis = [0] * len(impact_category)
#     for i in range(len(plot_x_axis)):
#         plot_x_axis[i] = impact_category[i][1]
#         #print(filtered_methods[i][1])

#     # Initialize an empty list to store the results
#     product_details = {}
#     product_details_code ={}

#     if process:
#         for proc in process:
#             # Initialize an empty list for the current process to store its product details
#             product_details[proc['name']] = []
#             product_details_code[proc['name']] = []

#             # Loop through the exchanges
#             for exc in proc.exchanges():
                
#                 # We're looking for technosphere exchanges, which are the inputs to the process
#                 #print(exc['type'], exc.input['code'])
#                 if exc['type'] == 'technosphere':
#                     #if  proc['name'] in exc.input['name']:
#                     product_details[proc['name']].append({exc.input['name'] : [exc['amount'], exc.input]})
#                     sub_proc = []
#                     if exc.input in eidb:# or exc.input in eidb_consq:
#                         # print(exc)
#                         product_details_code[proc['name']].append([exc.input['name'],exc.input['code']])
  
                            

#     idx_df = []
#     fu_val = []
#     p_code = []

#     # Print or save the extracted product details
#     for process_name, details in product_details.items():
#         for detail in range(len(details)):
#             for key, item in details[detail].items():
#                 idx_df.append(key)
#                 fu_val.append(details[detail][key][0])
#                 p_code.append(details[detail][key])

#     FU_proc = []

#     for flow in flows:
#         for flow_length in range(len(product_details[flow])):
#             for key in product_details[flow][flow_length].keys():
#                 # for flow in range(len(flows)):
#                 if flow in key:
#                     #print(key)
#                     key = key.replace(f'{flow} ', '')
#                 FU_proc.append(key)

#     FU = []
#     for key, item in product_details.items():
#         for idx in item:
#             for n, m in idx.items():
#                 FU.append({ key: {m[1]:m[0]}})
#                 #print(m)
#     print('Initialization is done')
#     return FU, FU_proc, impact_category, plot_x_axis, product_details_code

In [264]:
def LCA_initialization(name, db, flows):
    db_consq = 'Consequential'
    db_cyl = 'Cylinder'
    db_pellet = 'Pellet'
    bd.projects.set_current(name)

    bi.bw2setup()
    eidb = bd.Database(db)
    eidb_consq = bd.Database(db_consq)
    eidb_cyl = bd.Database(db_cyl)
    eidb_pellet = bd.Database(db_pellet)

    # Corrected dictionary comprehension
    procces_keys = {key: None for key in flows}

    size = len(flows)
    all_acts = list(eidb) + list(eidb_consq) + list(eidb_cyl) + list(eidb_pellet)

    # Find process keys in eidb or eidb_consq
    for act in all_acts:
        for proc in range(size):
            if act['name'] == flows[proc]:
                procces_keys[flows[proc]] = act['code']

    process = []
    key_counter = 0

    # Loop through processes and copy from eidb_consq if missing from eidb
    for key, item in procces_keys.items():
        try:
            if eidb.get(item) in eidb:
                # If process is found in eidb, use it
                process.append(eidb.get(item))
                print(f'Process "{key}" found in main database (eidb): {eidb.get(item)}')
            else:
                # If process is missing in eidb, copy from eidb_consq
                copied_process = copy_process(item, eidb_consq, eidb)
                if copied_process:
                    process.append(copied_process)
                    print('Process copied')
                else:
                    print(f"Process with key '{item}' not found in the consequential database (eidb_consq) either.")
        except KeyError:
            print(f"Process with key '{item}' not found in the database '{eidb}'")
            process = None
        key_counter += 1

    products_list = []
    
    if process:
        for proc in process:
            # Loop through the exchanges and filter for products
            for exc in proc.exchanges():
                if exc['type'] == 'production':
                    # Add the product (output) to the list
                    products_list.append(exc.input)


    # Initialize an empty list to store the linked processes
    linked_processes_list = []

    if process:
        for proc in process:
            # Loop through the exchanges and extract linked process keys
            for exc in proc.exchanges():
                # Add the linked process (input) to the list
                linked_processes_list.append(exc.input)

            # Remove duplicates by converting to a set and back to a list
            linked_processes_list = list(set(linked_processes_list))

        # Display the list of linked processes
        proc_keys = {}
        name_keys = {}

        for linked_process in linked_processes_list:
            # Initialize the list for this database if it doesn't exist
            if linked_process[0] not in proc_keys:
                proc_keys[linked_process[0]] = []
                name_keys[linked_process[0]] = []
            # Append the process key to the list
            proc_keys[linked_process[0]].append(linked_process[1])
            name_keys[linked_process[0]].append(linked_process)

    # List all methods containing 'EF v3.1 EN15804'
    all_methods = [m for m in bw.methods if 'EF v3.1 EN15804' in str(m)]

    # Filter out methods that contain "climate change:" in method[1]
    filtered_methods = [method for method in all_methods if "climate change:" not in method[1]]

    # Print the methods that were removed
    removed_methods = [method[1] for method in all_methods if "climate change:" in method[1]]

    impact_category = filtered_methods
    
    plot_x_axis = [0] * len(impact_category)
    for i in range(len(plot_x_axis)):
        plot_x_axis[i] = impact_category[i][1]
        #print(filtered_methods[i][1])

    # Initialize an empty list to store the results
    product_details = {}
    product_details_code = {}

    if process:
        for proc in process:
            # Initialize an empty list for the current process to store its product details
            product_details[proc['name']] = []
            product_details_code[proc['name']] = []

            # Loop through the exchanges
            for exc in proc.exchanges():
                # if exc['type'] == 'biosphere' and 'Use phase' in exc.output['name']:
                #     print('Oi', exc, exc['type'])
                # We're looking for technosphere exchanges, which are the inputs to the process
                #print(exc['type'], exc.input['code'])
                if 'Use' in exc.output['name'] and exc['type'] == 'biosphere':
                    product_details[proc['name']].append({exc.input['name'] : [exc['amount'], exc.input]})
                    #sub_proc = []
                    if exc.input in eidb or exc.input in eidb_consq or exc.input in eidb_cyl:
                        # print(exc)
                        product_details_code[proc['name']].append([exc.input, exc.input['name'],exc.input['code'], exc['amount'] ])
                elif exc['type'] == 'technosphere':
                    #if  proc['name'] in exc.input['name']:
                    product_details[proc['name']].append({exc.input['name'] : [exc['amount'], exc.input]})
                    #sub_proc = []
                    if exc.input in eidb or exc.input in eidb_consq or exc.input in eidb_cyl:
                        # print(exc)
                        product_details_code[proc['name']].append([exc.input, exc.input['name'],exc.input['code'], exc['amount'] ])
  
                            

    idx_df = []
    fu_val = []
    p_code = []

    # Print or save the extracted product details
    for process_name, details in product_details.items():
        for detail in range(len(details)):
            for key, item in details[detail].items():
                idx_df.append(key)
                fu_val.append(details[detail][key][0])
                p_code.append(details[detail][key])

    FU_proc = []

    for flow in flows:
        for flow_length in range(len(product_details[flow])):
            for key in product_details[flow][flow_length].keys():
                # for flow in range(len(flows)):
                if flow in key:
                    #print(key)
                    key = key.replace(f'{flow} ', '')
                FU_proc.append(key)

    FU = []
    for key, item in product_details.items():
        for idx in item:
            for n, m in idx.items():
                FU.append({ key: {m[1]:m[0]}})
#                 #print(m)

    print('Initialization is done')
    return FU, FU_proc, impact_category, plot_x_axis, product_details_code


# Helper function to copy process from eidb_consq to eidb
def copy_process(process_code, eidb_consq, eidb):
    try:
        external_process = eidb_consq.get(process_code)
        if external_process:
            # Create a new process in eidb based on the external_process properties from eidb_consq
            new_process = eidb.new_activity(
                code=external_process['code'],
                name=external_process['name'],
                unit=external_process['unit'],
                location=external_process['location'],
            )
            # Copy all exchanges
            for exc in external_process.exchanges():
                new_process.new_exchange(
                    input=exc.input,
                    output=new_process,
                    type=exc['type'],
                    amount=exc['amount'],
                    unit=exc['unit']
                )
            new_process.save()
            print(f"Process '{external_process['name']}' copied from eidb_consq to eidb.")
            return new_process
    except Exception as e:
        print(f"Error copying process from eidb_consq: {e}")
    return None


In [265]:
def life_cycle_impact_assessment(flows, functional_unit, impact_categories, process):
    # Define the dimensions
    n = len(flows)  # number of rows (flows)
    m = len(impact_categories)  # number of columns (impact categories)

    # Create a DataFrame to store results
    df = pd.DataFrame(0, index=flows, columns=impact_categories, dtype=object)  # dtype=object to handle lists

    calc_count = 1
    row_counter = 0

    # Loop through impact categories
    for impact in range(m):
        # Loop through flows
        for f in flows:
            df_lst = []  # Clear list for each flow in each impact category
            for func_unit in range(len(functional_unit)):
                for FU_key, FU_item in functional_unit[func_unit].items():
                    cat = impact_categories[impact]
                    if f in FU_key:
                        # Perform LCA
                        lca = bw.LCA(FU_item, cat)
                        lca.lci()
                        lca.lcia()
                        if len(process) == 1:
                            df_lst.append([process, lca.score])
                        else:
                            df_lst.append([process[func_unit], lca.score])

                        # Print progress
                        print(f"Calculation {calc_count} of {m * len(functional_unit)}", FU_item, cat[1], lca.score)
                        calc_count += 1
            
            # Assign the list of results to the DataFrame
            df.iloc[row_counter, impact] = df_lst
            print(f'{cat[1]} at row {row_counter} col {impact} has been assigned the list {df_lst}')

        # Update the row counter
        row_counter += 1
        if row_counter == len(flows):  # Reset when all flows have been processed
            row_counter = 0

    return df


In [266]:
def life_cycle_impact_assessment_contribution(flows, functional_unit, impact_categories, process):
    # Define the dimensions
    n = len(flows)  # number of rows (flows)
    m = len(impact_categories)  # number of columns (impact categories)

    # Create a DataFrame to store results
    df = pd.DataFrame(0, index=flows, columns=impact_categories, dtype=object)  # dtype=object to handle lists

    calc_count = 1
    row_counter = 0

    # Loop through impact categories
    for impact in range(m):
        # Loop through flows
        for f in flows:
            df_lst = []  # Clear list for each flow in each impact category
            for func_unit in range(len(functional_unit)):
                for FU_key, FU_item in functional_unit[func_unit].items():
                    cat = impact_categories[impact]
                    if f in FU_key:
                        # Perform LCA
                        lca = bw.LCA(FU_item, cat)
                        lca.lci()
                        lca.lcia()
                        if len(process) == 1:
                            df_lst.append([process, lca.score])
                        else:
                            df_lst.append([process[func_unit], lca.score])

                        # Print progress
                        print(f"Calculation {calc_count} of {m * len(functional_unit)}", FU_item, cat[1], lca.score)
                        calc_count += 1
            
            # Assign the list of results to the DataFrame
            df.iloc[row_counter, impact] = df_lst
            print(f'{cat[1]} at row {row_counter} col {impact} has been assigned the list {df_lst}')

        # Update the row counter
        row_counter += 1
        if row_counter == len(flows):  # Reset when all flows have been processed
            row_counter = 0

    return df


In [267]:
def life_cycle_impact_assessment_sub(flows, functional_unit, impact_categories, process):
    # Define the dimensions
    n = len(flows)  # number of rows
    m = len(impact_categories)  # number of columns

    df = pd.DataFrame(0, index=flows, columns=impact_categories, dtype=object)  # dtype to handle lists
    calc_count = 1

    for impact in range(m):
        row_counter = 0  # Reset row_counter for each impact category
        for f in flows:
            df_lst = []
            for func_unit in range(len(functional_unit)):
                for FU_key, FU_item in functional_unit[func_unit].items():
                    if f in FU_key:
                        cat = impact_categories[impact]
                        lca = bw.LCA(FU_item, cat)
                        lca.lci()
                        lca.lcia()
                        df_lst.append([process[func_unit], lca.score])
                        print(f"Calculation {calc_count} of {m*len(functional_unit)}", FU_item, cat[1], lca.score)
                        calc_count += 1
                  #  print(f'{cat[1]} at row {row_counter} col {impact} has been assigned the list {df_lst}')
            if row_counter < n:
                df.iloc[row_counter, impact] = df_lst
            
            row_counter += 1

    return df

In [268]:
# Your name variable should be the name of the project.
name = "RA - Rune"
db = 'DU consequential final'

flows = ['sc1 - No DU', 'sc2 - CDU', 'sc3 - MDU']
flow_legend = ['No DU', 'CDU', 'MDU']

FU, FU_proc, impact_category, plot_x_axis, sub_product_details = LCA_initialization(name, db, flows)
# df = life_cycle_impact_assessment(flows, FU, impact_category, FU_proc)
# df.head()
print(FU)
print(FU_proc)
# print(type(flows), '\n', type(FU), '\n', type(impact_category), '\n', type(FU_proc))

Biosphere database already present!!! No setup is needed
Process "sc1 - No DU" found in main database (eidb): 'sc1 - No DU' (unit, GLO, None)
Process "sc2 - CDU" found in main database (eidb): 'sc2 - CDU' (unit, GLO, None)
Process "sc3 - MDU" found in main database (eidb): 'sc3 - MDU' (unit, GLO, None)
Initialization is done
[{'sc1 - No DU': {'Steel cylinder' (unit, GLO, None): 6.84931506849315e-06}}, {'sc1 - No DU': {'nitrous oxide production' (kilogram, RER, None): 0.867460675121784}}, {'sc1 - No DU': {'Use phase basecase' (unit, GLO, None): 1}}, {'sc1 - No DU': {'EoL 50L cylinder - FU' (unit, GLO, None): 6.84931506849315e-06}}, {'sc2 - CDU': {'CDU updated' (unit, GLO, None): 6.84931506849315e-06}}, {'sc2 - CDU': {'Steel cylinder' (unit, GLO, None): 6.84931506849315e-06}}, {'sc2 - CDU': {'nitrous oxide production' (kilogram, RER, None): 0.867460675121784}}, {'sc2 - CDU': {'Use phase CDU' (unit, GLO, None): 1}}, {'sc2 - CDU': {'EoL CDU' (unit, GLO, None): 6.84931506849315e-06}}, {'sc2

In [269]:
sub_proccess = {}
amount = {}
for key, details in sub_product_details.items():
    print(f"Process: {key}")
    lst_proc = []
    sub_proccess[key] = []
    # amount[key] = []
    # lst_amount = []
    for detail in details:
        if isinstance(detail, dict):  # Check if detail is a dictionary
            for sub_key, sub_item in detail.items():
                print(f"   Sub-process: {detail[0]}, Amount: {detail[2]}, Code: {detail[1]}")
                lst_proc.append([detail[1],detail[3]])
                # lst_amount.append[detail[3]]
        elif isinstance(detail, list):  # Handle case where detail is a list
            print(f"  Database: {detail[0]}, Name: {detail[1]} Amount: {detail[3]}, Code: {detail[2]}")
            lst_proc.append([detail[0], detail[1], detail[3]])
            amount[detail[1]] = []
            amount[detail[1]].append(detail[3])
            # lst_amount.append()
    sub_proccess[key] = lst_proc
    # amount.append(lst_amount)



Process: sc1 - No DU
  Database: 'Steel cylinder' (unit, GLO, None), Name: Steel cylinder Amount: 6.84931506849315e-06, Code: 636399b3dbf048749d45830443a98208_copy1
  Database: 'nitrous oxide production' (kilogram, RER, None), Name: nitrous oxide production Amount: 0.867460675121784, Code: 0ac12b74daa63eee0208bafe172f8312
  Database: 'Use phase basecase' (unit, GLO, None), Name: Use phase basecase Amount: 1, Code: beb85b7efe644c7cb9b07492f44ef107_copy1
  Database: 'EoL 50L cylinder - FU' (unit, GLO, None), Name: EoL 50L cylinder - FU Amount: 6.84931506849315e-06, Code: 50569900c2224d2faa5abc33a31b355b_copy6
Process: sc2 - CDU
  Database: 'CDU updated' (unit, GLO, None), Name: CDU updated Amount: 6.84931506849315e-06, Code: ec6fd94e3ecf44d3a2d609c85c900261_copy3
  Database: 'Steel cylinder' (unit, GLO, None), Name: Steel cylinder Amount: 6.84931506849315e-06, Code: 636399b3dbf048749d45830443a98208_copy1
  Database: 'nitrous oxide production' (kilogram, RER, None), Name: nitrous oxide pr

In [270]:
amount

{'Steel cylinder': [6.84931506849315e-06],
 'nitrous oxide production': [0.867460675121784],
 'Use phase basecase': [1],
 'EoL 50L cylinder - FU': [6.84931506849315e-06],
 'CDU updated': [6.84931506849315e-06],
 'Use phase CDU': [1],
 'EoL CDU': [6.84931506849315e-06],
 'MDU updated': [0.000136986301369863],
 'Use phase MDU': [1],
 'EoL MDU': [0.000136986301369863]}

In [271]:
# Initializing empty dictionaries to store the results
FU_sub = {key: [] for key in sub_proccess}
FU_sub_proc = {key: [] for key in sub_proccess}


for proc, sub_proc in sub_proccess.items():
    # print(key, item)
    for proc_idx in range(len(sub_proc)):
        #print(sub_proc[proc_idx])
        flow = [sub_proc[proc_idx][1]]
        db_proc = sub_proc[proc_idx][0][0]
        #print(flow, db_proc)
        fu, p, ic, pxa, kokos = LCA_initialization(name, db_proc,flow)
        FU_sub[proc].append(fu)
        FU_sub_proc[proc].append([flow, p])



Biosphere database already present!!! No setup is needed
Process "Steel cylinder" found in main database (eidb): 'Steel cylinder' (unit, GLO, None)
Initialization is done
Biosphere database already present!!! No setup is needed
Process "nitrous oxide production" found in main database (eidb): 'nitrous oxide production' (kilogram, RoW, None)
Initialization is done
Biosphere database already present!!! No setup is needed
Process "Use phase basecase" found in main database (eidb): 'Use phase basecase' (unit, GLO, None)
Initialization is done
Biosphere database already present!!! No setup is needed
Process "EoL 50L cylinder - FU" found in main database (eidb): 'EoL 50L cylinder - FU' (unit, GLO, None)
Initialization is done
Biosphere database already present!!! No setup is needed
Process "CDU updated" found in main database (eidb): 'CDU updated' (unit, GLO, None)
Initialization is done
Biosphere database already present!!! No setup is needed
Process "Steel cylinder" found in main database 

In [304]:
FU_sub_proc

{'sc1 - No DU': [[['Steel cylinder'],
   ['Compresser opration 50L',
    'prod.',
    'nitrous oxide production',
    'Hydrastic testing 50L',
    'transport, freight, lorry 16-32 metric ton, EURO6',
    'transport, freight, inland waterways, barge tanker']],
  [['nitrous oxide production'],
   ['market for ammonia, anhydrous, liquid',
    'market for ammonia, anhydrous, liquid',
    'market for ammonia, anhydrous, liquid',
    'market for ammonia, anhydrous, liquid',
    'market for ammonia, anhydrous, liquid',
    'market for ammonia, anhydrous, liquid',
    'market for ammonia, anhydrous, liquid',
    'market for ammonia, anhydrous, liquid',
    'market for chemical factory, organics',
    'market group for electricity, medium voltage',
    'market group for electricity, medium voltage',
    'market for electricity, medium voltage',
    'market group for electricity, medium voltage',
    'market for electricity, medium voltage',
    'market group for electricity, medium voltage',
  

In [294]:
df_sub = {} # life_cycle_impact_assessment(sub_proccess, FU_sub, impact_category, FU_sub_proc)
proc_counter = 1
flow_sub = []
functional_unit_sub = []
for key, item in FU_sub_proc.items():
    #print(key)
    df_temp = {}
    for pommesfrit in item:
        fu_proc_temp = pommesfrit[0][0]
        fu_sub_proc_temp = pommesfrit[1]
        fu_temp = FU_sub[key][1]
        #print(f' Process : {fu_proc_temp}\n flow : {fu_sub_proc_temp}\n FU : {fu_temp}')
        proc_counter += 1
        flow_sub.append(fu_proc_temp)
        functional_unit_sub.append(fu_temp)
n = proc_counter  # number of rows (flows)
m = len(impact_category)  # number of columns (impact categories)


272

In [301]:

    # Create a DataFrame to store results
df = pd.DataFrame(0, index=flow_sub, columns=impact_category, dtype=object)  # dtype=object to handle lists

calc_count = 1

dct = {}
# Loop through impact categories
for impact in range(m):
# Loop through flows
        row_counter = 0
        for f in flow_sub:
                if f not in flows:
                        # print('f is in flows')
                        dct[f] = []
                        df_lst = []  # Clear list for each flow in each impact category
                        for func_unit in range(len(functional_unit_sub)):
                                #print(len(functional_unit_sub[func_unit]))
                                for FU_dict in functional_unit_sub[func_unit]:
                                        
                                        for key, item in FU_dict.items():
                                                
                                                FU_dict_copy = copy.deepcopy(FU_dict)
                                                #print(f, item, calc_count)
                                                cat = impact_category[impact]
                                                
                                                dct[f].append(item)
                                                # a = item.get()
                                                for FU_key,FU_val in item.items():
                                                        #print(f'flow : {f}, FU : {FU_key}. FU value : {FU_val}')
                                                        FU_dict_copy[key][FU_key] = FU_dict[key][FU_key] * amount[f][0]
                                                        #print(FU_dict_copy[key][FU_key])
                                                        # print(FU_key)
                                                # lca = bw.LCA(FU_dict_copy[key], cat)
                                                # lca.lci()
                                                # lca.lcia()
                                                # df_lst.append([f'{FU_key}', lca.score])
                                                print(f"Calculation {calc_count} of {m*len(functional_unit_sub)}")#, FU_dict_copy[key], cat[1], lca.score)
                                #                 calc_count += 1
                                # if row_counter < n:
                                #         df.iloc[row_counter, impact] = df_lst
                                #         print(f'{cat[1]} at row {row_counter} col {impact} has been assigned the list {df_lst}')
                                # row_counter += 1
        

Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1 of 256
Calculation 1

In [280]:
df

,"(EF v3.1 EN15804, acidification, accumulated exceedance (AE))","(EF v3.1 EN15804, climate change, global warming potential (GWP100))","(EF v3.1 EN15804, ecotoxicity: freshwater, comparative toxic unit for ecosystems (CTUe))","(EF v3.1 EN15804, energy resources: non-renewable, abiotic depletion potential (ADP): fossil fuels)","(EF v3.1 EN15804, eutrophication: freshwater, fraction of nutrients reaching freshwater end compartment (P))","(EF v3.1 EN15804, eutrophication: marine, fraction of nutrients reaching marine end compartment (N))","(EF v3.1 EN15804, eutrophication: terrestrial, accumulated exceedance (AE))","(EF v3.1 EN15804, human toxicity: carcinogenic, comparative toxic unit for human (CTUh))","(EF v3.1 EN15804, human toxicity: non-carcinogenic, comparative toxic unit for human (CTUh))","(EF v3.1 EN15804, ionising radiation: human health, human exposure efficiency relative to u235)","(EF v3.1 EN15804, land use, soil quality index)","(EF v3.1 EN15804, material resources: metals/minerals, abiotic depletion potential (ADP): elements (ultimate reserves))","(EF v3.1 EN15804, ozone depletion, ozone depletion potential (ODP))","(EF v3.1 EN15804, particulate matter formation, impact on human health)","(EF v3.1 EN15804, photochemical oxidant formation: human health, tropospheric ozone concentration increase)","(EF v3.1 EN15804, water use, user deprivation potential (deprivation-weighted water consumption))"
Steel cylinder,"[['market for ammonia, anhydrous, liquid' (kil...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
nitrous oxide production,"[['market for ammonia, anhydrous, liquid' (kil...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Use phase basecase,"[['market for ammonia, anhydrous, liquid' (kil...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
EoL 50L cylinder - FU,"[['market for ammonia, anhydrous, liquid' (kil...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
CDU updated,"[['market for ammonia, anhydrous, liquid' (kil...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Steel cylinder,"[['market for ammonia, anhydrous, liquid' (kil...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
nitrous oxide production,"[['market for ammonia, anhydrous, liquid' (kil...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Use phase CDU,"[['market for ammonia, anhydrous, liquid' (kil...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
EoL CDU,"[['market for ammonia, anhydrous, liquid' (kil...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
EoL 50L cylinder - FU,"[['market for ammonia, anhydrous, liquid' (kil...",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [312]:
for uu, oo in FU_sub_proc.items():
    print(uu, oo)

sc1 - No DU [[['Steel cylinder'], ['Compresser opration 50L', 'prod.', 'nitrous oxide production', 'Hydrastic testing 50L', 'transport, freight, lorry 16-32 metric ton, EURO6', 'transport, freight, inland waterways, barge tanker']], [['nitrous oxide production'], ['market for ammonia, anhydrous, liquid', 'market for ammonia, anhydrous, liquid', 'market for ammonia, anhydrous, liquid', 'market for ammonia, anhydrous, liquid', 'market for ammonia, anhydrous, liquid', 'market for ammonia, anhydrous, liquid', 'market for ammonia, anhydrous, liquid', 'market for ammonia, anhydrous, liquid', 'market for chemical factory, organics', 'market group for electricity, medium voltage', 'market group for electricity, medium voltage', 'market for electricity, medium voltage', 'market group for electricity, medium voltage', 'market for electricity, medium voltage', 'market group for electricity, medium voltage', 'market for heat, from steam, in chemical industry', 'market for oxygen, liquid']], [['Use

In [319]:
# Create a deep copy to ensure changes don't affect the original
dct_copy = copy.deepcopy(dct)

# Iterate over the dictionary items
for key, item in FU_sub_proc.items():
    # print(key, len(item))
    for sub_item in range(len(item)):
        for quantity in FU_sub_proc[key][sub_item]:
            # Multiply the value by 2 and update the copy
            #ct_copy[key][sub_item][proc_] = dct[key][sub_item][proc_] * amount[key][0]
            print(f'key : {key}\n Item : {item}\n Sub item : {sub_item}\n Quanitity : {quantity}')
# dct_copy now contains the updated values
# dct_copy


key : sc1 - No DU
 Item : [[['Steel cylinder'], ['Compresser opration 50L', 'prod.', 'nitrous oxide production', 'Hydrastic testing 50L', 'transport, freight, lorry 16-32 metric ton, EURO6', 'transport, freight, inland waterways, barge tanker']], [['nitrous oxide production'], ['market for ammonia, anhydrous, liquid', 'market for ammonia, anhydrous, liquid', 'market for ammonia, anhydrous, liquid', 'market for ammonia, anhydrous, liquid', 'market for ammonia, anhydrous, liquid', 'market for ammonia, anhydrous, liquid', 'market for ammonia, anhydrous, liquid', 'market for ammonia, anhydrous, liquid', 'market for chemical factory, organics', 'market group for electricity, medium voltage', 'market group for electricity, medium voltage', 'market for electricity, medium voltage', 'market group for electricity, medium voltage', 'market for electricity, medium voltage', 'market group for electricity, medium voltage', 'market for heat, from steam, in chemical industry', 'market for oxygen, liq

In [ ]:
df_sub = {} # life_cycle_impact_assessment(sub_proccess, FU_sub, impact_category, FU_sub_proc)


In [ ]:
p = [sub_proccess['sc2 - CDU'][2][1]]
d = sub_proccess['sc2 - CDU'][2][0][0]
print(p,d)

In [ ]:

FU1, FU_proc1, impact_category1, plot_x_axis1, sub_product_details1 = LCA_initialization(name, d, p)

In [ ]:
for func in FU1:
    for key, item in func.items():
        # print(key, item)
        for k, i in item.items():
            print(k,i)

In [ ]:
for key, item in FU_sub_proc.items():
    print(key, item)

In [ ]:
df_sub = life_cycle_impact_assessment(sub_proccess, FU_sub, impact_category, FU_sub_proc)

In [ ]:
df_sub

In [ ]:
FU_sub = {key: [] for key in sub_proccess}
FU_sub_proc = {key: [] for key in sub_proccess}
impact_category_sub = {key: [] for key in sub_proccess}
plot_x_axis_sub = {key: [] for key in sub_proccess}
sub_product_details1 = {key: [] for key in sub_proccess}
for key in sub_proccess.keys():
    print(key)
    FU_sub[key], FU_sub_proc[key], impact_category_sub[key], plot_x_axis_sub[key], sub_product_details1[key] = LCA_initialization_subprocess(name, db, sub_proccess[key])
    

In [ ]:
sub_proccess['sc1 - No DU']
# a, b, c, d, e = LCA_initialization_subprocess(name, db, sub_proccess['sc1 - No DU'])

In [ ]:
df_sub

In [ ]:
# # Create a dictionary to store DataFrames
df_sub = {}

# # Create and store DataFrames in the dictionary
# for key in sub_proccess.keys():
#     df_sub[key] = life_cycle_impact_assessment_sub(sub_proccess[key], FU_sub[key], impact_category, FU_sub_proc[key])

key = flows[0]

df_sub[key] = life_cycle_impact_assessment_sub(sub_proccess[key], FU_sub[key], impact_category, FU_sub_proc[key])


In [ ]:
len(sub_product_details1) * len(impact_category)

In [ ]:
df_sub = life_cycle_impact_assessment(sub_proccess, FU_sub, impact_category_sub, FU_sub_proc)
df_sub.head()

In [ ]:
file_name = r'Results\Ananas - CONSQ.xlsx'
file_name_sub = r'Results\Ananas sub - CONSQ.xlsx'
sheet_name = 'Ananas'

In [ ]:

# Convert each cell to a JSON string for all columns
df_sub = df_sub.applymap(lambda x: json.dumps(x) if isinstance(x, list) else x)

# Save to Excel
with pd.ExcelWriter(file_name_sub) as writer:
    df_sub.to_excel(writer, sheet_name=sheet_name, index=False, header=True)

print('DataFrame with nested lists written to Excel successfully.')

In [ ]:
# Reading from Excel
df1 = pd.read_excel(file_name)
df1_sub = pd.read_excel(file_name_sub)

# Convert JSON strings back to lists for all columns
df1 = df1.applymap(lambda x: json.loads(x) if isinstance(x, str) and x.startswith('[') else x)
df1_sub = df1_sub.applymap(lambda x: json.loads(x) if isinstance(x, str) and x.startswith('[') else x)

df1 = df1.set_axis(flows)
df1.columns = impact_category


df1_sub = df1_sub.set_axis(flows)
df1_sub.columns = impact_category
print(f'{df1} \n {df1_sub}')

In [ ]:
df_updated = df1.copy()
df_tot = df_updated.copy()
df_comp = df_updated.copy()

for col in range(df_updated.shape[1]):  # Iterate over columns
    for row in range(df_updated.shape[0]):  # Iterate over rows
        tot = 0
        diff = 0
        for i in range(len(df_updated.iloc[row,col])):
            #print(df_updated.iloc[row,col][i][1])
            tot += df_updated.iloc[row,col][i][1]
            diff += df1_sub.iloc[row,col][i][1]
        df_tot.iloc[row,col] = tot
        if tot - diff == 0:
            df_comp.iloc[row,col] = True
        else:
            df_comp.iloc[row,col] = False
        # print('New row')
df_comp

In [ ]:
df_cols = df_tot.columns
df_cols = df_cols.to_list()

df_norm = pd.DataFrame().reindex_like(df_tot) #https://stackoverflow.com/questions/23195250/create-empty-dataframe-with-same-dimensions-as-another
for i in df_cols:
    scaling_factor = max(abs(df_tot[i]))
    for j in range(len(df_tot[df_cols[0]])):
        df_norm[i][j] =df_tot[i][j]/scaling_factor
        #print(scaling_factor, i)
# for j in range(len(data['Index'])):
#     df_norm['Index'][j] = df['Index'][j]


# Selecting the columns from 1th column onwards
columns_to_plot = df_norm.columns
# df_cols

In [ ]:
index_list = list(df1.index.values)
plot_x_axis = [0] * len(impact_category)
for i in range(len(plot_x_axis)):
    if "photochemical oxidant formation" in impact_category[i][1]:
        plot_x_axis[i] = "photochemical oxidant formation"
    else:
        plot_x_axis[i] = impact_category[i][1].title()
    print(plot_x_axis[i])

df_GWP = df_updated[df_updated.columns[1]].to_frame()
df_GWP_sub = df1_sub[df_updated.columns[1]].to_frame()
df_GWP_sub

In [ ]:
for key, item in df_GWP_sub.items():
    print(item)
    for n in range(len(item)):
        print(len(item[n]))
        for m in range(len(item[n])):
            print(item[n][m])

In [ ]:
x_axis = []
GWP_value = []

# gwp_tot = [0] * len(df_updated)
for df_size in range(len(df_GWP)):
    lst_x = []
    lst_GWP = []
    gwp_tot = 0
    for key in df_GWP.keys():
        for lst_size in range(len(df_GWP[key][df_size])):
            #print(df_size, lst_size)
            x = df_GWP[key][df_size][lst_size][0]
            gwp = df_GWP[key][df_size][lst_size][1]
            
            print(key, gwp, x)
            if '- FU' in x:
                #print(key)
                x = x.replace(f' - FU', '')
            if 'Use' in x:           
                x = 'Labor'
            if 'nitrous' in x:
                x = 'N2O prod.'
            if 'market for electricity' in x:
                x = 'Avoided electricity'
            if 'Steel' in x:
                x = 'Cylinder prod.'

            if '50L' in x:
                x = x.replace(' 50L', '')
            if 'updated' in x:
                x = x.replace('updated', 'prod.')

            lst_x.append(x)
            lst_GWP.append(gwp)
            gwp_tot += gwp
    
    lst_GWP.append(gwp_tot)
    lst_x.append('Total')
    x_axis.append(lst_x)
    GWP_value.append(lst_GWP)

# x_axis

In [ ]:
# Categories definitions
categories = ["Raw mat. + prod.", "Use", "EoL", "Total"]
category_mapping = {
    "Raw mat. + prod.": ["Cylinder prod.", "N2O prod.", "CDU prod.", "MDU prod.", "Production"],
    "Use": ["Labor"],
    "EoL": ["EoL cylinder", "EoL CDU", "EoL MDU"],
    "Total": ["Total"]
}

# Ensure the legend displays items in the category order
ordered_legend = {key: [] for key in category_mapping}

for x_lst in range(len(x_axis)):
    for x in range(len(x_axis[x_lst])):
        
        for key, item in category_mapping.items():
                #print(x_axis[x_lst][x], item, x_axis[x_lst][x] in item)
                if x_axis[x_lst][x] in item:
                    # print(x_axis[x_lst][x], item, x_lst, x)
                    ordered_legend[key].append(x_axis[x_lst][x])

plot_legend = {key: [] for key in category_mapping}
temp = []

for key,value in ordered_legend.items():
    #print(key, value)
    for val in value:
        if val not in temp:
            temp.append(val)
            print(val)
            plot_legend[key].append(val)


colors = plot_colors(temp,'turbo')
len(temp)

In [ ]:
columns_to_plot = df_norm.columns

In [ ]:


# Plotting
fig, ax = plt.subplots(figsize=(9,7))

num_processes = len(df_norm)
bar_width = 1/(len(index_list) + 1.5) 
index = np.arange(len(columns_to_plot))



count = 3

# Plotting each group of bars
for i, process in enumerate(df_norm.index):
    values = df_norm.loc[process, columns_to_plot].values
    ax.bar((index + i * bar_width), values, bar_width, label=process, color=colors[i])
    count += 3
    

# Setting labels and title
ax.set_title('Scaled impact of the Functional Unit',weight='bold')
ax.set_xticks(index + bar_width )
ax.set_xticklabels(plot_x_axis)
plt.xticks(rotation=90)
plt.yticks(np.arange(0, 1.01, step=0.1))
plt.ylim(0,1.03)

ax.legend(flow_legend,bbox_to_anchor=(1.01, .75, .13, 0), loc="lower left",
            mode="expand", borderaxespad=0,  ncol=1)
plt.tight_layout()
# plt.savefig(os.path.join(save_dir, img_res), bbox_inches='tight')
plt.show()

In [ ]:
directory = 'C:\\Users\\ruw\\Desktop\\RA\\'
file = f'{directory}Single-use-vs-multi-use-in-health-care\\Norm + Weigh.xlsx'

data_NW = pd.read_excel(file)


In [ ]:
def LCA_normalization(file, columns, df):
    # file = r'D:\UNI\MILJOETEK\4. semester\Calculations - Copy\LCIA FU_.xlsx'

    # sheet2 = 'Norm + Weigh'

    data_NW = pd.read_excel(file)

    norm_lst = data_NW['Normalization']
    weigh_lst = data_NW['Weighting'].to_list
    #print(len(data_NW['Normalization']))

    norm_lst = [0]*(len(data_NW['Normalization']))
    weigh_lst = [0]*(len(data_NW['Weighting']))

    for i in range(len(data_NW['Normalization'])):
        norm_lst[i] = data_NW['Normalization'][i]
        weigh_lst[i] = data_NW['Weighting'][i]

    norm_df = pd.DataFrame().reindex_like(df) #https://stackoverflow.com/questions/23195250/create-empty-dataframe-with-same-dimensions-as-another
    weigh_df = pd.DataFrame().reindex_like(df)

    counter = 0
    for i in columns:
        for j in range(len(df[columns[0]])):
            norm_df[i][j] =df[i][j] * norm_lst[counter]
            weigh_df[i][j] =norm_df[i][j] * weigh_lst[counter]
            #print(i, norm_df[i][j], weigh_df[i][j])
        counter +=1

    lst = [0] * len(weigh_df[columns[0]])
    for j in range(len(weigh_df[columns[0]])):
        for i in columns:
            lst[j] += weigh_df[i][j]
            #print(i,j)

    lst_norm_weighted = [0] * len(lst)
    lst_max = max(lst)


    for n in range(len(lst)):
        lst_norm_weighted[n] = lst[n]/lst_max

    # lst_scaled
    return lst_norm_weighted

In [ ]:
lst_scaled = LCA_normalization(file, df_cols, df_tot)

In [ ]:
# Plotting
fig, ax = plt.subplots(figsize=(9,7))

num_processes = len(df_norm)
bar_width = 1/(len(index_list)-1) 
index = np.arange(len(index_list))   

ax.bar(index + bar_width, lst_scaled, bar_width, label=index_list, color=colors)

# Setting labels and title
ax.set_title('Scaled single score of the Functional Unit',weight='bold')
ax.set_xticks(index + bar_width )
ax.set_xticklabels(flow_legend)
#plt.xticks(rotation=90)
plt.yticks(np.arange(0, 1.01, step=0.1))
plt.ylim(0,1.03)

plt.tight_layout()
# plt.savefig(os.path.join(save_dir, img_res), bbox_inches='tight')
plt.show()

In [ ]:
# Calculate bottom and top parts
bottoms = lst_scaled[1:]
last_value = lst_scaled[0]
tops = [last_value - bottom for bottom in bottoms]

# Plotting
fig, ax = plt.subplots(figsize=(6, 4))

# Bar positions
bar_positions = [0, 1.75]

# Bottom parts (first two values)
ax.bar(bar_positions, bottoms, label='Emissions',color=colors[0], edgecolor=colors[0])

# Top parts (values after subtraction) with hatching
ax.bar(bar_positions, tops, bottom=bottoms, label='Avoided emissions', color=colors[1], edgecolor=colors[2], hatch='/')


# Adding labels and titlecolors[1]
ax.set_xticks(bar_positions)
ax.set_xticklabels(['CDU', 'MDU'])

# Setting labels and title
ax.set_title('Scaled single point score for the FU',weight='bold')
plt.yticks(np.arange(0, 1.02, step=0.1))
plt.ylim(0, 1.02)

ax.legend(['Emissions', 'Avoided emissions'], bbox_to_anchor=(0.18, -0.18, 0.6, 0.2), loc="lower left",
            mode="expand", borderaxespad=0,  ncol=2)

# ax.plot(bottoms,'o',color=colors[1])
# Show the plot
plt.tight_layout()
#plt.savefig(os.path.join(save_dir, 'GWP_res.jpg'), bbox_inches='tight')
plt.show()

In [ ]:

color_map = {}
#unique_processes = {process for sublist in x_axis for process in sublist}
for i, process in enumerate(temp):
    color_map[process] = colors[i]
    #print(process, i)



# Initialize an ordered dictionary for legend_handles to maintain the order
legend_handles = OrderedDict()

# Initialize legend_handles with keys from plot_legend and empty lists
for process in temp:
    legend_handles[process] = None

# Plotting logic
if len(x_axis) == len(GWP_value):
    num_scenarios = len(GWP_value)  # Number of scenarios
    bar_width = 0.15  # Width of the bars for each scenario
    space_between_scenarios = 0.05  # Space between each scenario set
    index = np.arange(len(categories))  # X-axis index positions for the categories

    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 6))

    idx = ['x', '^', 'o']  # List of markers for scenarios
    all_markers = []  # List to store Line2D objects for markers

    # Main plotting logic
    for scenario in range(num_scenarios):
        bottom_positive = np.zeros(len(categories))  # Initialize the bottom array for positive values
        bottom_negative = np.zeros(len(categories))  # Initialize the bottom array for negative values
        scenario_index = index + scenario * (bar_width + space_between_scenarios)

        for length in range(len(x_axis[scenario])):
            process_name = x_axis[scenario][length]
            value = GWP_value[scenario][length]

            # Determine which category this process falls into
            for i, category in enumerate(categories):
                if any(keyword in process_name for keyword in category_mapping[category]):
                    # Assign color based on the process name
                    color = color_map[process_name]

                    # Create a bar with the specific color
                    if value >= 0:
                        bar = ax.bar(scenario_index[i], value, bar_width,
                                    label=f"{process_name}" if legend_handles[process_name] is None else "",
                                    bottom=bottom_positive[i],
                                    color=color)
                        bottom_positive[i] += value
                    else:
                        bar = ax.bar(scenario_index[i], value, bar_width,
                                    label=f"{process_name}" if legend_handles[process_name] is None else "",
                                    bottom=bottom_negative[i],
                                    color=color)
                        bottom_negative[i] += value

                    # Add the bar to the corresponding process in legend_handles
                    if legend_handles[process_name] is None:
                        legend_handles[process_name] = bar

                    # Add plot markers (symbols) at the bottom
                    ax.plot(scenario_index[i], -5, marker=idx[scenario], color='gray')

                    break

    # Add custom markers to the legend
    for i, marker in enumerate(idx):
        all_markers.append(Line2D([0], [0], marker=marker, color='gray', label=f'Scenario {i + 1}', linestyle='None'))

    # Set x-axis labels and ticks, adjusting to account for spacing
    tick_positions = index + (num_scenarios - 1) * (bar_width + space_between_scenarios) / 2
    ax.set_xticks(tick_positions)
    ax.set_xticklabels(categories)

    # Axis limits
    # ax.set_ylim(-.45, 3.55)
    # ax.set_yticks(np.arange(-0.5, 3.6, step=0.25))
    ax.set_ylabel("Global Warming Potential [kg CO$_2$e]")
    ax.set_title('GWP impact for each life stage for 1 FU')

    # Add markers to legend_handles for display at the bottom of the legend
    valid_legend_handles = [(k, v) for k, v in legend_handles.items() if v is not None]
    legend_handles_for_display = valid_legend_handles + [(flow_legend[i], marker) for i, marker in enumerate(all_markers)]

    if legend_handles_for_display:
        ax.legend(handles=[v for k, v in legend_handles_for_display], labels=[k for k, v in legend_handles_for_display], bbox_to_anchor=(1.005, 1), loc='upper left')

    plt.tight_layout()
    #plt.savefig(os.path.join(save_dir, f'LC_impact_ofir_APOS.jpg'), bbox_inches='tight')
    plt.show()

else:
    print('The x-axis and GWP values have different sizes')
